In [1]:
import random
import nufhe

In [2]:
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()

param = nufhe.PerformanceParameters(nufhe.NuFHEParameters(transform_type='FFT'), 
                                    ntt_base_method='cuda_asm', ntt_mul_method='cuda_asm', 
                                    ntt_lsh_method='cuda_asm',
                                    use_constant_memory_multi_iter=True,
                                    use_constant_memory_single_iter=True,
                                    transforms_per_block=8,
                                    single_kernel_bootstrap=True)

vm = ctx.make_virtual_machine(cloud_key, param)

# vm = ctx.make_virtual_machine(cloud_key)
size = 128

In [3]:
N_EXPERIMENTS = 100

In [4]:
%%time
for _ in range(0, N_EXPERIMENTS):
    bits1 = [random.choice([False, True]) for i in range(size)]
    bits2 = [random.choice([False, True]) for i in range(size)]

    ciphertext1 = ctx.encrypt(secret_key, bits1)
    ciphertext2 = ctx.encrypt(secret_key, bits2)

    reference = [not (b1 and b2) for b1, b2 in zip(bits1, bits2)]
    result = vm.gate_nand(ciphertext1, ciphertext2)
    result_bits = ctx.decrypt(secret_key, result)

    assert all(result_bits == reference)

CPU times: user 27.5 s, sys: 61.6 ms, total: 27.5 s
Wall time: 27.7 s
